[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AhmetSeyhan/scanner-ultra/blob/main/notebooks/train_colab.ipynb)

# 🛡️ Scanner ULTRA v5.0 — PentaShield™ Model Training

**Google Colab Enterprise / Colab Pro+ Training Notebook**

Bu notebook, Scanner ULTRA deepfake detection modellerini eğitmek için tasarlanmıştır.

## Modeller
| Model | Params | Dataset | Hedef AUC |
|-------|--------|---------|----------|
| EfficientNet-B0 | 5.3M | FF++ | >0.95 |
| CLIP ViT-L/14 (LayerNorm) | 428M (0.03% trainable) | FF++ | >0.93 |
| Xception | 22M | FF++ | >0.96 |
| ViT-B/16 | 86M | FF++ | >0.94 |

## GPU Gereksinimleri
- **Minimum:** T4 (16GB) — EfficientNet + CLIP
- **Önerilen:** A100 (40GB) — Tüm modeller paralel

---

## 1️⃣ GPU & Ortam Kontrolü

In [ ]:
import subprocess, sys

# GPU kontrolü
result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total,driver_version',
                         '--format=csv,noheader'], capture_output=True, text=True)
if result.returncode == 0:
    print('✅ GPU Bulundu:', result.stdout.strip())
else:
    print('⚠️  GPU bulunamadı — CPU ile devam edilecek (YAVAŞ)')

# Python & CUDA
import torch
print(f'PyTorch: {torch.__version__}')
print(f'CUDA: {torch.version.cuda}')
print(f'Device: {"cuda" if torch.cuda.is_available() else "cpu"}')
if torch.cuda.is_available():
    print(f'GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')

## 2️⃣ Google Drive Bağlantısı

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
# Çalışma dizinleri
DRIVE_ROOT = '/content/drive/MyDrive/Scanner'
os.makedirs(f'{DRIVE_ROOT}/weights', exist_ok=True)
os.makedirs(f'{DRIVE_ROOT}/datasets', exist_ok=True)
os.makedirs(f'{DRIVE_ROOT}/logs', exist_ok=True)
print(f'✅ Drive bağlandı: {DRIVE_ROOT}')

## 3️⃣ Repo Clone & Bağımlılıklar

In [ ]:
%%bash
# Repo klonla
if [ ! -d '/content/scanner-ultra' ]; then
    git clone https://github.com/AhmetSeyhan/scanner-ultra.git /content/scanner-ultra
    echo '✅ Repo klonlandı'
else
    cd /content/scanner-ultra && git pull
    echo '✅ Repo güncellendi'
fi

In [ ]:
%%bash
# Bağımlılıkları kur
pip install -q \
    timm>=0.9.12 \
    transformers>=4.36.0 \
    accelerate>=0.25.0 \
    librosa>=0.10.0 \
    soundfile>=0.12.0 \
    opencv-python-headless>=4.9.0 \
    scikit-learn>=1.3.0 \
    matplotlib>=3.7.0 \
    tqdm>=4.66.0
echo '✅ Bağımlılıklar kuruldu'

In [ ]:
import sys
sys.path.insert(0, '/content/scanner-ultra/src')
sys.path.insert(0, '/content/scanner-ultra/scripts/training')
print('✅ Path ayarlandı')

## 4️⃣ Dataset Hazırlama

### Seçenek A: FaceForensics++ (Önerilen)
### Seçenek B: Celeb-DF v2
### Seçenek C: Demo Dataset (Küçük test için)

> **NOT:** FaceForensics++ akademik lisans gerektirir: https://github.com/ondyari/FaceForensics

In [ ]:
# ============================================================
# SEÇENEK C: Demo Dataset (Hızlı test — gerçek veri yok)
# Kendi verini kullanmak için SEÇENEK A veya B'yi kullan
# ============================================================
import os
import numpy as np
from PIL import Image

DATASET_DIR = '/content/demo_dataset'
USE_DEMO = True  # Gerçek veri varsa False yap

if USE_DEMO:
    print('📁 Demo dataset oluşturuluyor...')
    for split in ['real', 'fake']:
        os.makedirs(f'{DATASET_DIR}/{split}', exist_ok=True)
    
    # 200 sahte görüntü oluştur (gerçek eğitimde gerçek veri kullan!)
    for i in range(100):
        # Real: düz renk + gürültü
        img = Image.fromarray(
            (np.random.rand(224, 224, 3) * 255).astype(np.uint8))
        img.save(f'{DATASET_DIR}/real/img_{i:04d}.jpg')
        
        # Fake: farklı dağılım
        img = Image.fromarray(
            (np.random.rand(224, 224, 3) * 128 + 64).astype(np.uint8))
        img.save(f'{DATASET_DIR}/fake/img_{i:04d}.jpg')
    
    print(f'✅ Demo dataset: {DATASET_DIR}')
    print('   real/: 100 görüntü, fake/: 100 görüntü')
    print('   ⚠️  Bu DEMO verisi — gerçek eğitim için FF++ kullan!')

else:
    # Gerçek dataset yolu
    # Google Drive'dan kopyala:
    # !cp -r '/content/drive/MyDrive/Scanner/datasets/FF++' /content/dataset
    DATASET_DIR = '/content/dataset'  # Buraya gerçek yolu gir
    print(f'Dataset: {DATASET_DIR}')

In [ ]:
# ============================================================
# SEÇENEK A: FaceForensics++ İndirme
# (download_script.py gerekli — FF++ sitesinden al)
# ============================================================
# FF++ video'larından frame çıkarma

def extract_ff_frames(video_dir, output_dir, n_frames=10):
    """FF++ video klasöründen frame çıkar."""
    import cv2
    from pathlib import Path
    
    video_dir = Path(video_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    count = 0
    for video_path in video_dir.rglob('*.mp4'):
        cap = cv2.VideoCapture(str(video_path))
        total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_indices = np.linspace(0, total-1, n_frames, dtype=int)
        
        for fi in frame_indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, fi)
            ret, frame = cap.read()
            if ret:
                out_path = output_dir / f'{video_path.stem}_f{fi:06d}.jpg'
                cv2.imwrite(str(out_path), frame)
                count += 1
        cap.release()
    
    print(f'✅ {count} frame çıkarıldı → {output_dir}')
    return count

# Kullanım:
# extract_ff_frames('/content/drive/MyDrive/FF++/original_sequences/youtube/c23/videos', 
#                   '/content/dataset/real')
# extract_ff_frames('/content/drive/MyDrive/FF++/manipulated_sequences/Deepfakes/c23/videos',
#                   '/content/dataset/fake')

print('Frame çıkarma fonksiyonu hazır (FF++ için)')

## 5️⃣ EfficientNet-B0 Eğitimi (Ana Model)

In [ ]:
import torch
import timm
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import numpy as np
from pathlib import Path
from PIL import Image
import time, logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s — %(message)s')
logger = logging.getLogger(__name__)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {DEVICE}')


class DeepfakeDataset(Dataset):
    REAL_DIRS = {'real', 'authentic', 'genuine', 'original'}
    FAKE_DIRS = {'fake', 'deepfake', 'manipulated', 'synthetic', 'generated'}
    IMG_EXTS = {'.jpg', '.jpeg', '.png', '.bmp', '.webp'}

    def __init__(self, data_dir, transform=None, split='train', val_ratio=0.15, seed=42):
        self.transform = transform
        self.samples = []
        for subdir in Path(data_dir).iterdir():
            if not subdir.is_dir(): continue
            name = subdir.name.lower()
            if name in self.REAL_DIRS: label = 0
            elif name in self.FAKE_DIRS: label = 1
            else: continue
            self.samples += [(p, label) for p in subdir.rglob('*')
                             if p.suffix.lower() in self.IMG_EXTS]
        rng = np.random.default_rng(seed)
        idx = rng.permutation(len(self.samples)).tolist()
        n_val = int(len(idx) * val_ratio)
        self.samples = [self.samples[i] for i in (idx[:n_val] if split == 'val' else idx[n_val:])]
        print(f'{split}: {len(self.samples)} samples '
              f'(real={sum(1 for _,l in self.samples if l==0)}, '
              f'fake={sum(1 for _,l in self.samples if l==1)})')

    def __len__(self): return len(self.samples)
    def __getitem__(self, i):
        path, label = self.samples[i]
        img = Image.open(path).convert('RGB')
        return self.transform(img) if self.transform else img, label


train_tf = transforms.Compose([
    transforms.Resize((256, 256)), transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
val_tf = transforms.Compose([
    transforms.Resize((224, 224)), transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

train_ds = DeepfakeDataset(DATASET_DIR, train_tf, 'train')
val_ds   = DeepfakeDataset(DATASET_DIR, val_tf, 'val')
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=64, shuffle=False, num_workers=2, pin_memory=True)
print('✅ DataLoader hazır')

In [ ]:
# Model
model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=2)
model = model.to(DEVICE)

# Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20, eta_min=1e-6)

EPOCHS = 20  # Gerçek veri için artır
WEIGHTS_OUT = f'{DRIVE_ROOT}/weights'

print(f'Model: EfficientNet-B0 | Params: {sum(p.numel() for p in model.parameters())/1e6:.1f}M')
print(f'Epochs: {EPOCHS} | Output: {WEIGHTS_OUT}')

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
import torch.nn.functional as F

def train_one_epoch(model, loader, criterion, optimizer):
    model.train()
    total_loss = 0
    for imgs, labels in loader:
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        loss = criterion(model(imgs), labels)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        total_loss += loss.item() * len(imgs)
    return total_loss / len(loader.dataset)

@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    all_preds, all_labels, all_probs = [], [], []
    for imgs, labels in loader:
        logits = model(imgs.to(DEVICE))
        probs = F.softmax(logits, dim=-1)[:, 1].cpu().numpy()
        preds = logits.argmax(-1).cpu().numpy()
        all_preds.extend(preds); all_labels.extend(labels.numpy()); all_probs.extend(probs)
    acc = accuracy_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs) if len(set(all_labels)) > 1 else 0.0
    f1  = f1_score(all_labels, all_preds, zero_division=0)
    return acc, auc, f1


best_auc = 0.0
history = []

print('🚀 EfficientNet-B0 Eğitimi Başlıyor...')
print('-' * 65)

for epoch in range(1, EPOCHS + 1):
    t0 = time.time()
    loss = train_one_epoch(model, train_loader, criterion, optimizer)
    acc, auc, f1 = evaluate(model, val_loader)
    scheduler.step()
    elapsed = time.time() - t0
    
    history.append({'epoch': epoch, 'loss': loss, 'acc': acc, 'auc': auc, 'f1': f1})
    print(f'Epoch {epoch:3d}/{EPOCHS} | loss={loss:.4f} acc={acc:.4f} auc={auc:.4f} f1={f1:.4f} | {elapsed:.0f}s')
    
    if auc > best_auc:
        best_auc = auc
        torch.save(model.state_dict(), f'{WEIGHTS_OUT}/best_efficientnet_b0.pth')
        print(f'  *** ✅ Yeni Best AUC={best_auc:.4f} — Drive\'e kaydedildi ***')

torch.save(model.state_dict(), f'{WEIGHTS_OUT}/last_efficientnet_b0.pth')
print(f'\n🎉 Eğitim tamamlandı! Best AUC={best_auc:.4f}')

In [ ]:
# Eğitim Grafiği
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(15, 4))
epochs = [h['epoch'] for h in history]

axes[0].plot(epochs, [h['loss'] for h in history], 'b-o', markersize=3)
axes[0].set_title('Training Loss'); axes[0].set_xlabel('Epoch'); axes[0].grid(True)

axes[1].plot(epochs, [h['auc'] for h in history], 'r-o', markersize=3)
axes[1].set_title('Val AUC'); axes[1].set_xlabel('Epoch'); axes[1].grid(True)
axes[1].axhline(0.95, color='g', linestyle='--', label='Hedef 0.95')
axes[1].legend()

axes[2].plot(epochs, [h['acc'] for h in history], 'g-o', markersize=3)
axes[2].plot(epochs, [h['f1'] for h in history], 'm-o', markersize=3)
axes[2].set_title('Accuracy & F1'); axes[2].set_xlabel('Epoch')
axes[2].legend(['Accuracy', 'F1']); axes[2].grid(True)

plt.tight_layout()
plt.savefig(f'{DRIVE_ROOT}/logs/efficientnet_training.png', dpi=150)
plt.show()
print('📊 Grafik kaydedildi')

## 6️⃣ CLIP ViT-L/14 Fine-Tuning (LayerNorm-Only)

In [ ]:
from transformers import CLIPModel, CLIPProcessor

print('📥 CLIP ViT-L/14 yükleniyor (428M parametre)...')
clip_processor = CLIPProcessor.from_pretrained('openai/clip-vit-large-patch14')
clip_model = CLIPModel.from_pretrained('openai/clip-vit-large-patch14')

# Freeze all — unfreeze only LayerNorm
for param in clip_model.parameters():
    param.requires_grad = False

n_trainable = 0
for name, param in clip_model.named_parameters():
    if 'layernorm' in name.lower() or 'layer_norm' in name.lower():
        param.requires_grad = True
        n_trainable += param.numel()

total_params = sum(p.numel() for p in clip_model.parameters())
print(f'✅ CLIP yüklendi')
print(f'   Total: {total_params/1e6:.1f}M params')
print(f'   Trainable (LayerNorm): {n_trainable:,} ({100*n_trainable/total_params:.3f}%)')

clip_model = clip_model.to(DEVICE)

In [ ]:
# Probe head (CLIP üstüne binary classifier)
clip_probe = nn.Sequential(
    nn.Linear(768, 256), nn.ReLU(), nn.Dropout(0.3), nn.Linear(256, 2)
).to(DEVICE)

def clip_collate(batch):
    imgs, labels = zip(*batch)
    inputs = clip_processor(images=list(imgs), return_tensors='pt', padding=True)
    return inputs, torch.tensor(labels, dtype=torch.long)

# Raw PIL images için dataset
class RawPILDataset(Dataset):
    def __init__(self, samples):
        self.samples = samples
    def __len__(self): return len(self.samples)
    def __getitem__(self, i):
        path, label = self.samples[i]
        return Image.open(path).convert('RGB'), label

train_ds2 = DeepfakeDataset(DATASET_DIR, transform=None, split='train')
val_ds2   = DeepfakeDataset(DATASET_DIR, transform=None, split='val')
# transform=None için PIL döndürmek üzere __getitem__ güncellenmeli
# (Demo dataset zaten PIL-compatible)

clip_train = DataLoader(
    [(Image.open(p).convert('RGB'), l) for p, l in train_ds2.samples[:500]],  # İlk 500
    batch_size=16, shuffle=True, collate_fn=clip_collate
)

clip_val = DataLoader(
    [(Image.open(p).convert('RGB'), l) for p, l in val_ds2.samples[:100]],
    batch_size=16, shuffle=False, collate_fn=clip_collate
)

print(f'✅ CLIP DataLoader: train={len(clip_train.dataset)}, val={len(clip_val.dataset)}')

In [ ]:
clip_params = (list(filter(lambda p: p.requires_grad, clip_model.parameters()))
               + list(clip_probe.parameters()))
clip_optimizer = optim.AdamW(clip_params, lr=5e-5, weight_decay=1e-4)
CLIP_EPOCHS = 10

best_clip_auc = 0.0
print('🚀 CLIP Fine-tuning Başlıyor...')

for epoch in range(1, CLIP_EPOCHS + 1):
    # Train
    clip_model.train(); clip_probe.train()
    total_loss = 0
    for inputs, labels in clip_train:
        inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
        labels = labels.to(DEVICE)
        clip_optimizer.zero_grad()
        features = clip_model.get_image_features(**inputs)
        features = features / (features.norm(dim=-1, keepdim=True) + 1e-8)
        loss = criterion(clip_probe(features), labels)
        loss.backward()
        clip_optimizer.step()
        total_loss += loss.item() * len(labels)
    
    # Eval
    clip_model.eval(); clip_probe.eval()
    all_preds, all_labels, all_probs = [], [], []
    with torch.no_grad():
        for inputs, labels in clip_val:
            inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
            features = clip_model.get_image_features(**inputs)
            features = features / (features.norm(dim=-1, keepdim=True) + 1e-8)
            logits = clip_probe(features)
            probs = F.softmax(logits, -1)[:, 1].cpu().numpy()
            all_preds.extend(logits.argmax(-1).cpu().numpy())
            all_labels.extend(labels.numpy())
            all_probs.extend(probs)
    
    auc = roc_auc_score(all_labels, all_probs) if len(set(all_labels)) > 1 else 0.0
    acc = accuracy_score(all_labels, all_preds)
    print(f'Epoch {epoch:2d}/{CLIP_EPOCHS} | loss={total_loss/len(clip_train.dataset):.4f} '
          f'acc={acc:.4f} auc={auc:.4f}')
    
    if auc > best_clip_auc:
        best_clip_auc = auc
        # LayerNorm ağırlıklarını kaydet (küçük dosya ~2MB)
        ln_state = {k: v for k, v in clip_model.state_dict().items()
                    if 'layernorm' in k.lower() or 'layer_norm' in k.lower()}
        torch.save(ln_state, f'{WEIGHTS_OUT}/clip_layernorm.pth')
        torch.save(clip_probe.state_dict(), f'{WEIGHTS_OUT}/clip_probe_head.pth')
        print(f'  *** ✅ Best CLIP AUC={best_clip_auc:.4f} kaydedildi ***')

print(f'\n🎉 CLIP Fine-tuning tamamlandı! Best AUC={best_clip_auc:.4f}')

## 7️⃣ Xception Eğitimi

In [ ]:
# Xception (pretrained via timm)
print('📥 Xception yükleniyor...')
xception = timm.create_model('xception', pretrained=True, num_classes=2).to(DEVICE)

xception_optimizer = optim.AdamW(xception.parameters(), lr=1e-4, weight_decay=1e-4)
xception_scheduler = optim.lr_scheduler.CosineAnnealingLR(xception_optimizer, T_max=15)

xception_tf = transforms.Compose([
    transforms.Resize((299, 299)),  # Xception 299x299 ister
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),  # Xception norm
])
xception_val_tf = transforms.Compose([
    transforms.Resize((299, 299)), transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
])

xc_train_ds = DeepfakeDataset(DATASET_DIR, xception_tf, 'train')
xc_val_ds   = DeepfakeDataset(DATASET_DIR, xception_val_tf, 'val')
xc_train_loader = DataLoader(xc_train_ds, batch_size=16, shuffle=True, num_workers=2)
xc_val_loader   = DataLoader(xc_val_ds, batch_size=32, shuffle=False, num_workers=2)

XCEPTION_EPOCHS = 15
best_xc_auc = 0.0
print('🚀 Xception Eğitimi Başlıyor...')

for epoch in range(1, XCEPTION_EPOCHS + 1):
    loss = train_one_epoch(xception, xc_train_loader, criterion, xception_optimizer)
    acc, auc, f1 = evaluate(xception, xc_val_loader)
    xception_scheduler.step()
    print(f'Epoch {epoch:2d}/{XCEPTION_EPOCHS} | loss={loss:.4f} acc={acc:.4f} auc={auc:.4f}')
    if auc > best_xc_auc:
        best_xc_auc = auc
        torch.save(xception.state_dict(), f'{WEIGHTS_OUT}/best_xception.pth')
        print(f'  *** ✅ Best Xception AUC={best_xc_auc:.4f} ***')

print(f'\n🎉 Xception tamamlandı! Best AUC={best_xc_auc:.4f}')

## 8️⃣ Sonuçlar & Model Karşılaştırması

In [ ]:
results = [
    {'Model': 'EfficientNet-B0', 'Best AUC': best_auc, 'Params': '5.3M', 'File': 'best_efficientnet_b0.pth'},
    {'Model': 'CLIP ViT-L/14 (LN-only)', 'Best AUC': best_clip_auc, 'Params': '0.03%', 'File': 'clip_layernorm.pth'},
    {'Model': 'Xception', 'Best AUC': best_xc_auc, 'Params': '22M', 'File': 'best_xception.pth'},
]

print('\n' + '='*60)
print('📊 EĞİTİM SONUÇLARI')
print('='*60)
for r in results:
    status = '✅' if r['Best AUC'] > 0.9 else '⚠️ '
    print(f"{status} {r['Model']:30s} | AUC={r['Best AUC']:.4f} | Params={r['Params']}")
print('='*60)
print(f'\n📁 Ağırlıklar kaydedildi: {WEIGHTS_OUT}')
import os
for f in os.listdir(WEIGHTS_OUT):
    size = os.path.getsize(f'{WEIGHTS_OUT}/{f}') / 1e6
    print(f'   {f}: {size:.1f} MB')

## 9️⃣ Modeli Scanner ULTRA'ya Entegre Et

In [ ]:
# Scanner ULTRA'ya yükleyip test et
import sys
sys.path.insert(0, '/content/scanner-ultra/src')

import asyncio
from scanner.core.visual.efficientnet_detector import EfficientNetDetector
from scanner.core.visual.clip_detector import CLIPDetector
from scanner.core.base_detector import DetectorInput
import cv2
import numpy as np

async def test_detectors():
    # EfficientNet
    eff_det = EfficientNetDetector(
        model_path=f'{WEIGHTS_OUT}/best_efficientnet_b0.pth'
    )
    await eff_det.load_model()
    
    # Test frame
    test_frame = (np.random.rand(224, 224, 3) * 255).astype(np.uint8)
    inp = DetectorInput(frames=[test_frame])
    result = await eff_det.detect(inp)
    print(f'EfficientNet: score={result.score:.4f}, confidence={result.confidence:.4f}, method={result.method}')
    
    # CLIP
    clip_det = CLIPDetector(
        model_path=f'{WEIGHTS_OUT}/clip_layernorm.pth'
    )
    await clip_det.load_model()
    result2 = await clip_det.detect(inp)
    print(f'CLIP: score={result2.score:.4f}, confidence={result2.confidence:.4f}, method={result2.method}')
    
    print('\n✅ Modeller Scanner ULTRA ile uyumlu!')

await test_detectors()

In [ ]:
# Google Cloud Storage'a yükle (opsiyonel)
# from google.cloud import storage
# 
# def upload_to_gcs(local_path, bucket_name, gcs_path):
#     client = storage.Client()
#     bucket = client.bucket(bucket_name)
#     blob = bucket.blob(gcs_path)
#     blob.upload_from_filename(local_path)
#     print(f'✅ GCS\'e yüklendi: gs://{bucket_name}/{gcs_path}')
# 
# BUCKET = 'scanner-ultra-weights'  # GCS bucket adın
# for fname in ['best_efficientnet_b0.pth', 'clip_layernorm.pth', 'best_xception.pth']:
#     fpath = f'{WEIGHTS_OUT}/{fname}'
#     if os.path.exists(fpath):
#         upload_to_gcs(fpath, BUCKET, f'weights/v5/{fname}')

print('💡 GCS yükleme için yukarıdaki kodu uncomment et')
print('   Önce: !pip install google-cloud-storage')
print('   Sonra: GCS bucket oluştur ve bucket adını gir')

## 🏁 Özet & Sonraki Adımlar

### Eğitilen Modeller
- ✅ `best_efficientnet_b0.pth` — EfficientNet-B0 binary classifier
- ✅ `clip_layernorm.pth` — CLIP LayerNorm weights
- ✅ `clip_probe_head.pth` — CLIP probe head
- ✅ `best_xception.pth` — Xception binary classifier

### Sonraki Adımlar
1. **Daha büyük dataset:** FaceForensics++ tam dataset ile yeniden eğit
2. **ViT eğitimi:** `timm.create_model('vit_base_patch16_224', ...)` 
3. **WavLM audio:** `scripts/training/train_audio.py` çalıştır
4. **API deploy:** `docker-compose up` ile production'a al
5. **Vertex AI:** Bu notebook'u Vertex AI Training Job'a dönüştür

### Ağırlıkları Production'a Aktar
```bash
# API'ye yükle
curl -X POST http://localhost:8000/v1/scan \
  -H 'X-API-Key: your-key' \
  -F 'file=@test_video.mp4' \
  -F 'weights_dir=/app/weights'
```